In [1]:
import sys
sys.path.append("../libmitigation")

In [2]:
from inv_sgs import InvSGS
from inv_s_sgs import InvSSGS
from inv_lm_sgs import InvLMSGS
from inv_s_lm_sgs import InvSLMSGS
from inv_s_lm0_sgs import InvSLM0SGS
from sgs_algorithm import sgs_algorithm

In [3]:
import sys, time
import math
import mpmath as mp
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#Preparing qiskit environment
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute
from qiskit import IBMQ
from qiskit import Aer
from scipy import optimize


/Users/yangbo/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/qiskit/__init__.py:67: DeprecationWarning: Using Qiskit with Python 3.6 is deprecated as of the 0.17.0 release. Support for running Qiskit with Python 3.6 will be removed in a future release.
  "future release.", DeprecationWarning)


In [5]:
# importing the noise models and mitigation models
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import depolarizing_error
import qiskit.ignis.mitigation as mit
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

In [6]:
nbit = 10
shots = 100

In [7]:
noise_model = NoiseModel()
for qi in range(nbit):
    if qi >= 0:
        read_err = ReadoutError([[0.90, 0.10],[0.10,0.90]])
        noise_model.add_readout_error(read_err, [qi])

In [8]:
cal_shots = 8192
qr = QuantumRegister(nbit)
mit_pattern = [[i] for i in range(nbit)]
meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')
job = execute(meas_calibs, backend=Aer.get_backend('qasm_simulator'), shots=cal_shots, noise_model=noise_model)
cal_results = job.result()
meas_fitter = TensoredMeasFitter(cal_results, mit_pattern=mit_pattern)

In [9]:
qc = QuantumCircuit(nbit)
qc.h(0)
for i in range(1,nbit):
    qc.cx(0,i)
qc.measure_all()

In [10]:
job = execute(qc, backend=Aer.get_backend("qasm_simulator"), noise_model=noise_model, shots=shots)

In [11]:
counts = job.result().get_counts()

In [12]:
counts

{'0100000000': 1,
 '1000000101': 1,
 '1100000000': 1,
 '1111101110': 1,
 '0000101100': 1,
 '1100101100': 1,
 '0000000001': 2,
 '0010000000': 2,
 '0000000010': 4,
 '1111111010': 1,
 '0101100000': 1,
 '1001000100': 1,
 '1101110111': 1,
 '1111111110': 2,
 '1011111111': 3,
 '1000010100': 1,
 '1111010111': 2,
 '0001000000': 2,
 '0000010000': 3,
 '0000001000': 3,
 '0001000001': 1,
 '1111011111': 1,
 '0000000110': 1,
 '1111111111': 16,
 '0111111111': 2,
 '1110111111': 3,
 '0000100100': 1,
 '0001100000': 1,
 '1111111001': 1,
 '1001101111': 1,
 '1001000010': 1,
 '0010010000': 1,
 '1111101111': 2,
 '1101111111': 2,
 '1111111011': 2,
 '0100100000': 1,
 '0000110000': 1,
 '0010000100': 1,
 '0000100000': 1,
 '1000100000': 1,
 '0001000101': 1,
 '0000000100': 1,
 '0111001010': 1,
 '1111111101': 3,
 '1000000001': 1,
 '1000000000': 2,
 '1010101111': 1,
 '0000000000': 15}

In [13]:
meas_filter = InvSGS(nbit, meas_fitter.cal_matrices, mit_pattern)
x_tilde = meas_filter.apply(counts)
x_tilde

strict inverse + SGS algorithm
sum of mitigated probability vector x: 1.0000000000000022
number of positive values:  535
main process: Done!


{'0001000101': 0.060677335103038674,
 '1000000101': 0.10676844798716487,
 '1101110111': 0.1475837680636815,
 '0111001010': 0.19094041499848785,
 '1100101100': 0.2511496248048067,
 '1001000010': 0.45500266920532223,
 '1001101111': 0.456635563683273,
 '1000010100': 0.4569393632142513,
 '1001000100': 0.47123059414195984,
 '1010101111': 0.495077827848334,
 '0000010000': 1.393578032184067,
 '1111111101': 1.5734024367232864,
 '0000001000': 1.6466278686127194,
 '1011111111': 1.7836634516736778,
 '1110111111': 1.9069953615544648,
 '1111010111': 3.6933248256527578,
 '0000000010': 4.895657744354459,
 '0000000000': 38.21684167011144,
 '1111111111': 41.7979030000828}

In [14]:
meas_filter = InvSSGS(nbit, meas_fitter.cal_matrices, mit_pattern)
x_tilde_s = meas_filter.apply(counts)
x_tilde_s

Restriction to labels of y + SGS algorithm
sum of mitigated probability vector x_s: 1.9647898730144737
number of positive values:  44
main process: Done!


{'0001000101': 0.060677335103038674,
 '1000000101': 0.10676844798716487,
 '1101110111': 0.1475837680636815,
 '0111001010': 0.19094041499848785,
 '1100101100': 0.2511496248048067,
 '1001000010': 0.45500266920532223,
 '1001101111': 0.456635563683273,
 '1000010100': 0.4569393632142513,
 '1001000100': 0.47123059414195984,
 '1010101111': 0.495077827848334,
 '0000010000': 1.393578032184067,
 '1111111101': 1.5734024367232864,
 '0000001000': 1.6466278686127194,
 '1011111111': 1.7836634516736778,
 '1110111111': 1.9069953615544648,
 '1111010111': 3.6933248256527578,
 '0000000010': 4.895657744354459,
 '0000000000': 38.21684167011144,
 '1111111111': 41.7979030000828}

In [15]:
meas_filter = InvLMSGS(nbit, meas_fitter.cal_matrices, mit_pattern)
x_tilde = meas_filter.apply(counts)
x_tilde

Lagrange Multiplier + SGS algorithm
sum of mitigated probability vector x: 1.0000000000000022
sum_of_x: 1.000479595774066
sum of mitigated probability vector x_hat: 0.9995213229238598
number of positive values:  534
main process: Done!


{'0001000101': 0.06067733187432797,
 '1000000101': 0.10676922999951696,
 '1101110111': 0.14758426926246881,
 '0111001010': 0.19093872008825696,
 '1100101100': 0.25115010174521396,
 '1001000010': 0.45500127392445017,
 '1001101111': 0.45663484737879406,
 '1000010100': 0.4569405192335372,
 '1001000100': 0.4712302719663098,
 '1010101111': 0.4950774760939022,
 '0000010000': 1.3935788067525687,
 '1111111101': 1.573403436274502,
 '0000001000': 1.6466276705791605,
 '1011111111': 1.7836633774185813,
 '1110111111': 1.9069961248568597,
 '1111010111': 3.693325030451311,
 '0000000010': 4.895656783262325,
 '0000000000': 38.21684156712284,
 '1111111111': 41.79790316171504}

In [16]:
meas_filter = InvSLMSGS(nbit, meas_fitter.cal_matrices, mit_pattern)
x_tilde = meas_filter.apply(counts)
x_tilde

Restriction to labels of y + Lagrange Multiplier + SGS algorithm
sum of mitigated probability vector x_s: 1.9647898730144737
sum of mitigated probability vector x_hat_s: 1.9647674095865217
number of positive values:  44
main process: Done!


{'0001000101': 0.06067733187432901,
 '1000000101': 0.106769229999518,
 '1101110111': 0.14758426926246987,
 '0111001010': 0.190938720088258,
 '1100101100': 0.251150101745215,
 '1001000010': 0.4550012739244512,
 '1001101111': 0.4566348473787951,
 '1000010100': 0.45694051923353823,
 '1001000100': 0.47123027196631084,
 '1010101111': 0.4950774760939032,
 '0000010000': 1.3935788067525698,
 '1111111101': 1.5734034362745029,
 '0000001000': 1.6466276705791614,
 '1011111111': 1.7836633774185824,
 '1110111111': 1.9069961248568608,
 '1111010111': 3.693325030451312,
 '0000000010': 4.895656783262326,
 '0000000000': 38.21684156712284,
 '1111111111': 41.79790316171504}

In [17]:
meas_filter = InvSLM0SGS(nbit, meas_fitter.cal_matrices, mit_pattern)
x_tilde = meas_filter.apply(counts)
x_tilde

Restriction to labels of y + Lagrange Multiplier + SGS algorithm
sum of mitigated probability vector x_s: 1.9647898730144737
sum of mitigated probability vector x_hat_s: 1.964767366473731
number of positive values:  44
main process: Done!


{'0001000101': 0.0606773318681305,
 '1000000101': 0.10676923150038861,
 '1101110111': 0.14758427022439313,
 '0111001010': 0.19093871683531183,
 '1100101100': 0.2511501026605776,
 '1001000010': 0.45500127124656237,
 '1001101111': 0.456634846004032,
 '1000010100': 0.45694052145222147,
 '1001000100': 0.4712302713479808,
 '1010101111': 0.49507747541880054,
 '0000010000': 1.393578808239161,
 '1111111101': 1.573403438192888,
 '0000001000': 1.6466276701990838,
 '1011111111': 1.7836633772760704,
 '1110111111': 1.9069961263218238,
 '1111010111': 3.693325030844366,
 '0000000010': 4.895656781417762,
 '0000000000': 38.21684156692518,
 '1111111111': 41.797903162025285}